<a href="https://colab.research.google.com/github/ENGRBertoli/webapp00_SITE/blob/main/primeiro_site_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os

# Caminho dos arquivos para salvar os dados
BALANCAS_FILE = "balancas_data.pkl"
CLIENTES_FILE = "clientes_data.pkl"

# Função para carregar os dados
def carregar_dados(filepath):
    if os.path.exists(filepath):
        with open(filepath, "rb") as file:
            return pickle.load(file)
    return []

# Função para salvar os dados
def salvar_dados(filepath, dados):
    with open(filepath, "wb") as file:
        pickle.dump(dados, file)

# Função para exibir gráfico de barras verticais com cores personalizadas
def Grafico_Barra_Status_Vertical(rotulos, quantias, cores, Largura=8, Altura=5, Titulo_Grafico='Gráfico de Barra', Titulo_x='Status', Titulo_y='Quantidade'):
    fig, ax = plt.subplots(figsize=(Largura, Altura))
    bars = ax.bar(rotulos, quantias, color=cores)
    ax.set_xlabel(Titulo_x)
    ax.set_ylabel(Titulo_y)
    ax.set_title(Titulo_Grafico)
    for bar, value in zip(bars, quantias):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{value}', ha='center', va='bottom')
    st.pyplot(fig)

# Função para exportar dados para Excel
def exportar_para_excel(data, filename):
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)
    return filename

# Carregar os dados das balanças e clientes
balancas = carregar_dados(BALANCAS_FILE)
clientes = carregar_dados(CLIENTES_FILE)

# Função principal do app
def main():
    st.set_page_config(
        page_title="Painel de Gestão",
        page_icon="⚖️",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    st.title("Painel de Gestão de Fabricação e Clientes")

    # Menu lateral
    menu = st.sidebar.radio(
        "Escolha uma seção:",
        ["Adicionar Balança", "Listar Balanças", "Gráficos", "Gerenciar Clientes", "Exportar Dados"]
    )

    if menu == "Adicionar Balança":
        st.subheader("Adicionar uma nova balança")
        modelo = st.text_input("Modelo da balança:").strip()
        quantidade = st.number_input("Quantidade total (deixe em branco para 'Indisponível'):", min_value=0, step=1, value=0)

        if st.button("Adicionar"):
            balanca_existente = next((b for b in balancas if b["modelo"].lower() == modelo.lower()), None)

            if balanca_existente:
                balanca_existente["quantidade_total"] += quantidade
                if quantidade > 0:
                    balanca_existente["status_quantidades"]["Em produção"] += quantidade
                st.success(f"A quantidade foi atualizada para o modelo '{modelo}'. Nova quantidade total: {balanca_existente['quantidade_total']}.")
            else:
                nova_balanca = {
                    "modelo": modelo,
                    "quantidade_total": quantidade,
                    "status_quantidades": {
                        "Em produção": quantidade if quantidade > 0 else 0,
                        "Indisponível": 0 if quantidade > 0 else 1,
                        "Disponível para venda": 0,
                        "Vendido": 0
                    }
                }
                balancas.append(nova_balanca)
                if quantidade > 0:
                    st.success(f"Balança '{modelo}' adicionada com quantidade total de {quantidade}.")
                else:
                    st.warning(f"Balança '{modelo}' adicionada como 'Indisponível'.")

            salvar_dados(BALANCAS_FILE, balancas)

    elif menu == "Listar Balanças":
        st.subheader("Balanças cadastradas")
        if not balancas:
            st.warning("Nenhuma balança cadastrada.")
        else:
            for balanca in balancas:
                st.markdown(f"### Modelo: {balanca['modelo']}")
                st.write(f"**Quantidade Total:** {balanca['quantidade_total']}")

                for status, quantidade in balanca["status_quantidades"].items():
                    nova_quantidade = st.number_input(
                        f"Quantidade em '{status}' para {balanca['modelo']}:",
                        min_value=0,
                        value=quantidade,
                        step=1,
                        key=f"{balanca['modelo']}_{status}"
                    )
                    balanca["status_quantidades"][status] = nova_quantidade

                salvar_dados(BALANCAS_FILE, balancas)
                st.success(f"As quantidades para '{balanca['modelo']}' foram atualizadas.")

    elif menu == "Gráficos":
        st.subheader("Gráficos das balanças")
        if not balancas:
            st.warning("Nenhuma balança cadastrada para exibir nos gráficos.")
        else:
            for balanca in balancas:
                st.markdown(f"### Gráfico de Status para {balanca['modelo']}")
                Grafico_Barra_Status_Vertical(
                    list(balanca["status_quantidades"].keys()),
                    list(balanca["status_quantidades"].values()),
                    cores=["blue", "red", "green", "orange"],
                    Largura=10,
                    Altura=7,
                    Titulo_Grafico=f"Status do Modelo {balanca['modelo']}"
                )

    elif menu == "Gerenciar Clientes":
        st.subheader("Gerenciar Clientes")

        cliente_menu = st.radio("Opções:", ["Adicionar Cliente", "Listar e Editar Clientes", "Buscar Cliente por Número"])

        if cliente_menu == "Adicionar Cliente":
            st.subheader("Adicionar Cliente")
            nome = st.text_input("Nome do Cliente:")
            cnpj = st.text_input("CNPJ:")
            telefone = st.text_input("Telefone:")
            email = st.text_input("E-mail:")
            endereco = st.text_area("Endereço:")

            if st.button("Cadastrar Cliente"):
                novo_cliente = {
                    "numero": len(clientes) + 1,
                    "nome": nome,
                    "cnpj": cnpj,
                    "telefone": telefone,
                    "email": email,
                    "endereco": endereco
                }
                clientes.append(novo_cliente)
                salvar_dados(CLIENTES_FILE, clientes)
                st.success(f"Cliente '{nome}' cadastrado com sucesso!")

        elif cliente_menu == "Listar e Editar Clientes":
            st.subheader("Clientes Cadastrados")
            if not clientes:
                st.warning("Nenhum cliente cadastrado.")
            else:
                for cliente in clientes:
                    st.markdown(f"### Cliente #{cliente['numero']}")
                    cliente["nome"] = st.text_input(f"Nome (Cliente #{cliente['numero']})", value=cliente["nome"])
                    cliente["cnpj"] = st.text_input(f"CNPJ (Cliente #{cliente['numero']})", value=cliente["cnpj"])
                    cliente["telefone"] = st.text_input(f"Telefone (Cliente #{cliente['numero']})", value=cliente["telefone"])
                    cliente["email"] = st.text_input(f"E-mail (Cliente #{cliente['numero']})", value=cliente["email"])
                    cliente["endereco"] = st.text_area(f"Endereço (Cliente #{cliente['numero']})", value=cliente["endereco"])

                salvar_dados(CLIENTES_FILE, clientes)
                st.success("Dados dos clientes atualizados!")

        elif cliente_menu == "Buscar Cliente por Número":
            st.subheader("Buscar Cliente")
            numero_cliente = st.number_input("Número do Cliente:", min_value=1, step=1)
            cliente = next((c for c in clientes if c["numero"] == numero_cliente), None)

            if cliente:
                st.markdown(f"### Cliente #{cliente['numero']}")
                st.write(f"**Nome:** {cliente['nome']}")
                st.write(f"**CNPJ:** {cliente['cnpj']}")
                st.write(f"**Telefone:** {cliente['telefone']}")
                st.write(f"**E-mail:** {cliente['email']}")
                st.write(f"**Endereço:** {cliente['endereco']}")
            else:
                st.warning("Cliente não encontrado.")

    elif menu == "Exportar Dados":
        st.subheader("Exportar Dados para Excel")

        if st.button("Exportar Produtos"):
            arquivo_produtos = exportar_para_excel(balancas, "produtos.xlsx")
            st.success("Arquivo 'produtos.xlsx' gerado com sucesso!")
            st.download_button("Baixar Produtos", data=open(arquivo_produtos, "rb"), file_name="produtos.xlsx")

        if st.button("Exportar Clientes"):
            arquivo_clientes = exportar_para_excel(clientes, "clientes.xlsx")
            st.success("Arquivo 'clientes.xlsx' gerado com sucesso!")
            st.download_button("Baixar Clientes", data=open(arquivo_clientes, "rb"), file_name="clientes.xlsx")

if __name__ == "__main__":
    main()








